In [ ]:
import pandas as pd
import re


fidelidades_iniciais = [round(0.5 + 0.01 * i, 2) for i in range(51)]  
fidelidades_requisicoes = [round(0.51 + 0.01 * i, 2) for i in range(50)]  

# DataFrame preenchido com "x" quando a requisição <= fidelidade inicial
data = [['x' if request <= fidelity else '' for request in fidelidades_requisicoes] for fidelity in fidelidades_iniciais]
df = pd.DataFrame(data, index=fidelidades_iniciais, columns=fidelidades_requisicoes)

# Colunas e o índice
df.index.name = "Fidelidade Inicial"
df.columns.name = "Requisição"

# Arquivo de texto
with open("Canais Bitflip symmetric de 0.5 ate 1.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

# Capturando os valores
initial_fidelity_pattern = re.compile(r"Initial Fidelity:\s([\d.]+)")
request_pattern = re.compile(r"Request:\s([\d.]+)")
success_rate_pattern = re.compile(r"Porcentagem de sucesso:\s([\d.]+)%")

# Dicionário para armazenar os dados extraídos
success_rates = {}

# Variáveis para armazenar valores extraídos
current_initial_fidelity = None
current_request = None

for line in lines:
    initial_match = initial_fidelity_pattern.search(line)
    request_match = request_pattern.search(line)
    success_match = success_rate_pattern.search(line)
    
    if initial_match:
        current_initial_fidelity = float(initial_match.group(1))
    
    if request_match:
        current_request = float(request_match.group(1))
    
    if success_match and current_initial_fidelity is not None and current_request is not None:
        success_rate = float(success_match.group(1))
        success_rates[(current_initial_fidelity, current_request)] = success_rate

# Planilha com os valores extraídos 
for (initial_fidelity, request), success_rate in success_rates.items():
    if initial_fidelity in df.index and request in df.columns:
        df.at[initial_fidelity, request] = f"{success_rate:.2f}%"

# Salvando a planilha
df.to_excel("Valores de 0.5 até 1 Canais Bitflip.xlsx")
